In [1]:
from pedalboard import Pedalboard, Reverb, load_plugin, VST3Plugin
from pedalboard.io import AudioFile
from mido import Message # not part of Pedalboard, but convenient!
from utils import *

In [2]:
instrument = load_plugin("C:\\Program Files\\Common Files\\vst3\\Serum2.vst3\\Contents\\x86_64-win\\Serum2.vst3", plugin_name="Serum 2")

In [49]:
def filter_cat_params(params):
    cont_params = {}
    cat_params = {}
    for k, v in params.items():
        if re.match(r'^[abc]_warp(_2)?$', k):
            cont_params[k] = v
            continue
        if "mode" in k or "type" in k:
            cat_params[k] = v
            continue
        if len(v.valid_values) < 100:
            if 'semitones' in k or 'rate' in k or v.type == float:
                if v.range == (0.0, 1.0, 1.0):
                    cat_params[k] = v
                else:
                    cont_params[k] = v
            else:
                cat_params[k] = v
        else:
            cont_params[k] = v
    return cat_params, cont_params

In [50]:
target_params = filter_osc_params(instrument)
cat_params, cont_params = filter_cat_params(target_params)

In [52]:
print(len(target_params))
print(len(cat_params))
print(len(cont_params))

227
43
184


In [57]:
def get_value_range(param):
    if re.match(r'^[ABC] Warp( 2)?$', param.name):
        return 0.0, 1.0, 0.01
    valid_values = param.valid_values
    valid_raw_values = []
    for v in valid_values:
        if param.type == float:
            valid_raw_values.append(param.get_raw_value_for(v))
        elif param.type == str:
            valid_raw_values.append(param.get_raw_value_for(str(v)))
        elif param.type == bool:
            valid_raw_values.append(param.get_raw_value_for(v))
    min_raw, max_raw = min(valid_raw_values), max(valid_raw_values)
    step = (max_raw - min_raw) / (len(valid_values) - 1)
    return min_raw, max_raw, step

In [58]:
ranges = {}
cont_types = {}
for k, v in cont_params.items():
    if None in v.range:
        ranges[k] = get_value_range(v)
        cont_types[k] = v.type
    else:
        ranges[k] = v.range
        cont_types[k] = v.type
categories = {}
cat_types = {}
num_categories = {}
for k, v in cat_params.items():
    if len(v.valid_values) == 1:
        continue
    categories[k] = v.valid_values
    num_categories[k] = len(v.valid_values)
    cat_types[k] = v.type

In [59]:
def generate_random_param_vector(ranges, num_categories):
    param_vector = {}
    for k, v in ranges.items():
        param_vector[k] = random.uniform(v[0], v[1])
    for k, v in num_categories.items():
        param_vector[k] = random.randint(0, v - 1)
    return param_vector
generate_random_param_vector(ranges, num_categories)

{'a_level': 0.47660447130800543,
 'a_pan': 0.3772654716245303,
 'a_octave_oct': 1.9265662463902116,
 'a_semi_semitones': -10.08327377306913,
 'a_fine_cents': 32.652405373828145,
 'a_ratio': 0.7355817554244405,
 'a_hz_offset_hz': -22.60952450927084,
 'a_coarse_pitch': 0.7451162327021965,
 'a_start': 31.157495646901246,
 'a_end': 69.7015519618545,
 'a_scan_rate': 118.94634867214279,
 'a_scan_bpm_rate': 0.7709037972387492,
 'a_position': 77.86892831453574,
 'a_loop_start': 95.69302681168969,
 'a_loop_end': 8.029672169581758,
 'a_loop_x_fade': 84.68763677715194,
 'a_single_slice': 5.60621068244856,
 'a_unison': 15.52294127553482,
 'a_uni_detune': 0.8754376971876573,
 'a_uni_blend': 92.25805166390921,
 'a_uni_width': 64.9211987699195,
 'a_uni_span': 88.40669392980925,
 'a_uni_rand_start': 81.87379285406973,
 'a_uni_warp': -83.46846431027741,
 'a_uni_warp_2': -3.137400261927212,
 'a_warp': 0.7454603054880227,
 'a_warp_var': 0.9556526093438205,
 'a_warp_2': 0.7021077310781592,
 'a_warp_2_var'

In [60]:
config = {
    "ranges": ranges,
    "num_categories": num_categories,
    "cont_types": str(cont_types),
    "cat_types": str(cat_types)
}
import json
conf_file = "config.json"
with open(conf_file, "w") as f:
    json.dump(config, f)
